<a href="https://colab.research.google.com/github/Niranjan1422/Infosys_Internship/blob/main/Infosys_Data_Ingestion_Process_Chat%26Support_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries (Colab usually has them, but safe to run)
!pip install pandas numpy scikit-learn fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 95.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import logging
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from fuzzywuzzy import fuzz
import hashlib

In [ ]:
#1️⃣ Data Ingestion

In [3]:
FILE_PATH = "/content/drive/MyDrive/Infosys_Datasets_Proj/customer_support_tickets.csv"

try:
    df_raw = pd.read_csv(FILE_PATH)
    print("Data loaded successfully")
except Exception as e:
    print("Failed to load data:", e)

df_raw.head()

Data loaded successfully


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [ ]:
#2️⃣ Data Cleaning

In [6]:
#1) Handling Missing Values
df = df_raw.copy()

# Drop rows where critical fields are missing
critical_cols = ["Ticket ID", "Customer Name", "Ticket Type"]
df = df.dropna(subset=critical_cols)

# Fill non-critical missing values
df.fillna({
    "Ticket Priority": "Unknown",
    "Ticket Status": "Open"
}, inplace=True)

In [7]:
#2) Remove Duplicate Records
df = df.drop_duplicates()

In [9]:
# Standardize text columns 3) Fix Formatting & Typos
text_cols = ["Ticket Type", "Ticket Priority", "Ticket Status"]

for col in text_cols:
    df[col] = df[col].str.strip().str.title()

In [ ]:
#3️⃣ Data Validation

In [11]:
#1) Schema Validation

expected_columns = {
    "Ticket ID": int
    # "customer_id": int  # This column does not exist in the DataFrame, if it was intended to be generated, it should be handled before this step.
}

for col, dtype in expected_columns.items():
    # Check if the column exists before attempting to convert its type
    if col in df.columns:
        df[col] = df[col].astype(dtype, errors="ignore")
    else:
        print(f"Warning: Column '{col}' not found in DataFrame. Skipping type conversion.")

In [13]:
# Ensure Ticket ID is positive 2) Range & Mandatory Checks
df = df[df["Ticket ID"] > 0]

In [ ]:
#4️⃣ Data Transformation

In [15]:
#1) Data Type Conversion

df["Date of Purchase"] = pd.to_datetime(df["Date of Purchase"], errors="coerce")
df["Time to Resolution"] = pd.to_datetime(df["Time to Resolution"], errors="coerce")

In [17]:
#2) Derived Fields

df["resolution_time_hours"] = (
    df["Time to Resolution"] - df["Date of Purchase"]
).dt.total_seconds() / 3600

In [19]:
#3) Encoding (Categorical → Numeric)

df = pd.get_dummies(df, columns=["Ticket Priority", "Ticket Status"], drop_first=True)

In [20]:
#4) Normalization

scaler = StandardScaler()
df["resolution_time_hours_scaled"] = scaler.fit_transform(
    df[["resolution_time_hours"]].fillna(0)
)

In [ ]:
#5️⃣ Data Filtering

In [21]:
# Remove unnecessary columns
drop_cols = ["customer_email", "customer_phone"]
df.drop(columns=drop_cols, errors="ignore", inplace=True)

# Filter only valid tickets
df = df[df["resolution_time_hours"] >= 0]

In [ ]:
#6️⃣ Data Enrichment

In [22]:
#1) Add SLA Flag

df["sla_breached"] = df["resolution_time_hours"] > 48

/tmp/ipython-input-4138800362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sla_breached"] = df["resolution_time_hours"] > 48


In [24]:
#2) Issue Severity Mapping

severity_map = {
    "Login Issue": "Medium",
    "Payment Failure": "High",
    "Account Hacked": "Critical"
}

df["issue_severity"] = df["Ticket Type"].map(severity_map).fillna("Low")

In [ ]:
#7️⃣ Data Deduplication

In [26]:
#1) Primary-Key Based

df = df.drop_duplicates(subset=["Ticket ID"])

In [27]:
#2) Fuzzy Deduplication (Optional)

def is_similar(a, b, threshold=90):
    return fuzz.ratio(a, b) >= threshold

In [ ]:
#8️⃣ Data Masking & Security

In [28]:
#Hash PII Fields

def hash_value(val):
    if pd.isna(val):
        return val
    return hashlib.sha256(val.encode()).hexdigest()

if "customer_email" in df_raw.columns:
    df["customer_email_hashed"] = df_raw["customer_email"].apply(hash_value)

In [ ]:
#9️⃣ Data Standardization

In [30]:
# Standard date format
df["created_date"] = df["Date of Purchase"].dt.strftime("%Y-%m-%d")

# Canonical values
df["sla_breached"] = df["sla_breached"].map({True: "Y", False: "N"})

In [ ]:
#🔟 Error Handling & Logging

In [31]:
logging.basicConfig(
    filename="data_ingestion.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Data ingestion and preprocessing completed successfully")

In [ ]:
#1️⃣1️⃣ Metadata Handling

In [32]:
df["ingestion_timestamp"] = datetime.utcnow()
df["source_system"] = "Kaggle_Customer_Support"
df["data_version"] = "v1.0"

/tmp/ipython-input-4275386237.py:1: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df["ingestion_timestamp"] = datetime.utcnow()


In [ ]:
#1️⃣2️⃣ Sampling

In [33]:
# Random sample for testing
sample_df = df.sample(frac=0.1, random_state=42)

In [34]:
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,...,Ticket Priority_Medium,Ticket Status_Open,Ticket Status_Pending Customer Response,resolution_time_hours_scaled,sla_breached,issue_severity,created_date,ingestion_timestamp,source_system,data_version
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical Issue,Network problem,I'm facing a problem with my {product_purchase...,...,False,False,False,1.766082,Y,Low,2020-07-14,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing Inquiry,Account access,I'm having an issue with the {product_purchase...,...,False,False,False,1.482315,Y,Low,2020-11-13,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing Inquiry,Data loss,I'm having an issue with the {product_purchase...,...,False,False,False,2.138684,Y,Low,2020-02-04,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
10,11,Joseph Moreno,mbrown@example.org,48,Male,Nintendo Switch,2021-01-19,Cancellation Request,Data loss,I'm having an issue with the {product_purchase...,...,False,False,False,1.327127,Y,Low,2021-01-19,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
11,12,Brandon Arnold,davisjohn@example.net,51,Male,Microsoft Xbox Controller,2021-10-24,Product Inquiry,Software bug,I'm having an issue with the {product_purchase...,...,False,False,False,0.684977,Y,Low,2021-10-24,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0


In [35]:
df.to_csv("/content/drive/MyDrive/Infosys_Datasets_Proj/customer_support_tickets.csv", index=False)


In [36]:
print()
display()
df.head()
df.info()
df.shape


<class 'pandas.core.frame.DataFrame'>
Index: 2769 entries, 2 to 8467
Data columns (total 28 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   Ticket ID                                2769 non-null   int64         
 1   Customer Name                            2769 non-null   object        
 2   Customer Email                           2769 non-null   object        
 3   Customer Age                             2769 non-null   int64         
 4   Customer Gender                          2769 non-null   object        
 5   Product Purchased                        2769 non-null   object        
 6   Date of Purchase                         2769 non-null   datetime64[ns]
 7   Ticket Type                              2769 non-null   object        
 8   Ticket Subject                           2769 non-null   object        
 9   Ticket Description                       2769

(2769, 28)

In [37]:
import pandas as pd

df_raw = pd.read_csv("/content/drive/MyDrive/Infosys_Datasets_Proj/customer_support_tickets.csv")

print("Dataset Loaded")
print("Shape:", df_raw.shape)

df_raw.head()

Dataset Loaded
Shape: (2769, 28)


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,...,Ticket Priority_Medium,Ticket Status_Open,Ticket Status_Pending Customer Response,resolution_time_hours_scaled,sla_breached,issue_severity,created_date,ingestion_timestamp,source_system,data_version
0,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical Issue,Network problem,I'm facing a problem with my {product_purchase...,...,False,False,False,1.766082,Y,Low,2020-07-14,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
1,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing Inquiry,Account access,I'm having an issue with the {product_purchase...,...,False,False,False,1.482315,Y,Low,2020-11-13,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
2,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing Inquiry,Data loss,I'm having an issue with the {product_purchase...,...,False,False,False,2.138684,Y,Low,2020-02-04,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
3,11,Joseph Moreno,mbrown@example.org,48,Male,Nintendo Switch,2021-01-19,Cancellation Request,Data loss,I'm having an issue with the {product_purchase...,...,False,False,False,1.327127,Y,Low,2021-01-19,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
4,12,Brandon Arnold,davisjohn@example.net,51,Male,Microsoft Xbox Controller,2021-10-24,Product Inquiry,Software bug,I'm having an issue with the {product_purchase...,...,False,False,False,0.684977,Y,Low,2021-10-24,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0


In [39]:
df = df_raw.copy()

df = df.dropna(subset=["Ticket ID"])
df = df.drop_duplicates()

print("After Cleaning")
print("Shape:", df.shape)

df.head()

After Cleaning
Shape: (2769, 28)


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,...,Ticket Priority_Medium,Ticket Status_Open,Ticket Status_Pending Customer Response,resolution_time_hours_scaled,sla_breached,issue_severity,created_date,ingestion_timestamp,source_system,data_version
0,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical Issue,Network problem,I'm facing a problem with my {product_purchase...,...,False,False,False,1.766082,Y,Low,2020-07-14,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
1,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing Inquiry,Account access,I'm having an issue with the {product_purchase...,...,False,False,False,1.482315,Y,Low,2020-11-13,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
2,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing Inquiry,Data loss,I'm having an issue with the {product_purchase...,...,False,False,False,2.138684,Y,Low,2020-02-04,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
3,11,Joseph Moreno,mbrown@example.org,48,Male,Nintendo Switch,2021-01-19,Cancellation Request,Data loss,I'm having an issue with the {product_purchase...,...,False,False,False,1.327127,Y,Low,2021-01-19,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
4,12,Brandon Arnold,davisjohn@example.net,51,Male,Microsoft Xbox Controller,2021-10-24,Product Inquiry,Software bug,I'm having an issue with the {product_purchase...,...,False,False,False,0.684977,Y,Low,2021-10-24,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0


In [40]:
print("Column Names:")
print(df.columns.tolist())

Column Names:
['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Product Purchased', 'Date of Purchase', 'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Resolution', 'Ticket Channel', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating', 'resolution_time_hours', 'Ticket Priority_High', 'Ticket Priority_Low', 'Ticket Priority_Medium', 'Ticket Status_Open', 'Ticket Status_Pending Customer Response', 'resolution_time_hours_scaled', 'sla_breached', 'issue_severity', 'created_date', 'ingestion_timestamp', 'source_system', 'data_version']


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2769 entries, 0 to 2768
Data columns (total 28 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Ticket ID                                2769 non-null   int64  
 1   Customer Name                            2769 non-null   object 
 2   Customer Email                           2769 non-null   object 
 3   Customer Age                             2769 non-null   int64  
 4   Customer Gender                          2769 non-null   object 
 5   Product Purchased                        2769 non-null   object 
 6   Date of Purchase                         2769 non-null   object 
 7   Ticket Type                              2769 non-null   object 
 8   Ticket Subject                           2769 non-null   object 
 9   Ticket Description                       2769 non-null   object 
 10  Resolution                               2769 no

In [43]:
df["Date of Purchase"] = pd.to_datetime(df["Date of Purchase"], errors="coerce")
df["Time to Resolution"] = pd.to_datetime(df["Time to Resolution"], errors="coerce")

print("Null dates after conversion:")
print(df[["Date of Purchase", "Time to Resolution"]].isna().sum())

Null dates after conversion:
Date of Purchase      0
Time to Resolution    0
dtype: int64


In [45]:
df["resolution_time_hours"] = (
    df["Time to Resolution"] - df["Date of Purchase"]
).dt.total_seconds() / 3600

df[["Date of Purchase", "Time to Resolution", "resolution_time_hours"]].head()

,Date of Purchase,Time to Resolution,resolution_time_hours
0,2020-07-14,2023-06-01 18:05:38,25266.093889
1,2020-11-13,2023-06-01 01:57:40,22321.961111
2,2020-02-04,2023-06-01 19:53:42,29131.895000
3,2021-01-19,2023-05-31 23:51:49,20711.863611
4,2021-10-24,2023-06-01 09:27:51,14049.464167


In [47]:
df_encoded = pd.get_dummies(df, columns=["Customer Gender", "Ticket Type"], drop_first=True)

print("Encoded Shape:", df_encoded.shape)
df_encoded.head()

Encoded Shape: (2769, 32)


,Ticket ID,Customer Name,Customer Email,Customer Age,Product Purchased,Date of Purchase,Ticket Subject,Ticket Description,Resolution,Ticket Channel,...,created_date,ingestion_timestamp,source_system,data_version,Customer Gender_Male,Customer Gender_Other,Ticket Type_Cancellation Request,Ticket Type_Product Inquiry,Ticket Type_Refund Request,Ticket Type_Technical Issue
0,3,Christopher Robbins,gonzalestracy@example.com,48,Dell XPS,2020-07-14,Network problem,I'm facing a problem with my {product_purchase...,Case maybe show recently my computer follow.,Social media,...,2020-07-14,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,False,True,False,False,False,True
1,4,Christina Dillon,bradleyolson@example.org,27,Microsoft Office,2020-11-13,Account access,I'm having an issue with the {product_purchase...,Try capital clearly never color toward story.,Social media,...,2020-11-13,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,False,False,False,False,False,False
2,5,Alexander Carroll,bradleymark@example.com,67,Autodesk AutoCAD,2020-02-04,Data loss,I'm having an issue with the {product_purchase...,West decision evidence bit.,Email,...,2020-02-04,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,False,False,False,False,False,False
3,11,Joseph Moreno,mbrown@example.org,48,Nintendo Switch,2021-01-19,Data loss,I'm having an issue with the {product_purchase...,Measure tonight surface feel forward.,Phone,...,2021-01-19,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,True,False,True,False,False,False
4,12,Brandon Arnold,davisjohn@example.net,51,Microsoft Xbox Controller,2021-10-24,Software bug,I'm having an issue with the {product_purchase...,Measure there house management pick knowledge ...,Chat,...,2021-10-24,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,True,False,False,True,False,False


In [48]:
import logging

logging.basicConfig(
    filename="data_ingestion.log",
    level=logging.INFO
)

logging.info("Pipeline completed")

In [49]:
!cat data_ingestion.log

cat: data_ingestion.log: No such file or directory


In [50]:
print("Final Dataset")
print(df_encoded.shape)

df_encoded.sample(5)

Final Dataset
(2769, 32)


,Ticket ID,Customer Name,Customer Email,Customer Age,Product Purchased,Date of Purchase,Ticket Subject,Ticket Description,Resolution,Ticket Channel,...,created_date,ingestion_timestamp,source_system,data_version,Customer Gender_Male,Customer Gender_Other,Ticket Type_Cancellation Request,Ticket Type_Product Inquiry,Ticket Type_Refund Request,Ticket Type_Technical Issue
937,2905,Megan Lewis,sherylgoodman@example.com,52,Google Nest,2020-08-15,Battery life,I'm facing a problem with my {product_purchase...,Throughout general certainly number agreement.,Social media,...,2020-08-15,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,True,False,False,False,True,False
524,1597,Natalie Smith,donald23@example.net,31,LG OLED,2021-04-13,Cancellation request,I've noticed a software bug in the {product_pu...,Past design group computer discussion.,Email,...,2021-04-13,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,True,False,False,False,False,True
2001,6133,Tina Washington,philip89@example.org,45,Roomba Robot Vacuum,2021-11-05,Account access,I'm facing issues logging into my {product_pur...,Interest dinner federal although theory of.,Email,...,2021-11-05,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,False,True,True,False,False,False
2657,8135,Andrew Palmer,lauramorrison@example.com,58,Google Pixel,2021-11-05,Installation support,My {product_purchased} is making strange noise...,Hot green almost by course peace fall.,Phone,...,2021-11-05,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,False,False,False,False,True,False
2596,7945,Roy Boyd,gjackson@example.com,22,Amazon Echo,2020-09-19,Software bug,I'm having an issue with the {product_purchase...,Hot my reduce compare attention avoid need body.,Social media,...,2020-09-19,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0,False,False,False,False,False,False


In [51]:
print("STEP NAME")
print("Shape:", df.shape)
df.head()

STEP NAME
Shape: (2769, 28)


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,...,Ticket Priority_Medium,Ticket Status_Open,Ticket Status_Pending Customer Response,resolution_time_hours_scaled,sla_breached,issue_severity,created_date,ingestion_timestamp,source_system,data_version
0,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical Issue,Network problem,I'm facing a problem with my {product_purchase...,...,False,False,False,1.766082,Y,Low,2020-07-14,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
1,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing Inquiry,Account access,I'm having an issue with the {product_purchase...,...,False,False,False,1.482315,Y,Low,2020-11-13,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
2,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing Inquiry,Data loss,I'm having an issue with the {product_purchase...,...,False,False,False,2.138684,Y,Low,2020-02-04,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
3,11,Joseph Moreno,mbrown@example.org,48,Male,Nintendo Switch,2021-01-19,Cancellation Request,Data loss,I'm having an issue with the {product_purchase...,...,False,False,False,1.327127,Y,Low,2021-01-19,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0
4,12,Brandon Arnold,davisjohn@example.net,51,Male,Microsoft Xbox Controller,2021-10-24,Product Inquiry,Software bug,I'm having an issue with the {product_purchase...,...,False,False,False,0.684977,Y,Low,2021-10-24,2026-01-08 12:15:45.733923,Kaggle_Customer_Support,v1.0


In [52]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)